In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('mach2.0').getOrCreate()
from pyspark.ml.regression import LinearRegression
df=sqlContext.sql("SELECT * FROM ship")

In [2]:
df.head(1)

Out[ 2 ]: [Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [3]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [4]:
df.columns

Out[ 4 ]: 
['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [5]:
df.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)

In [6]:
from pyspark.ml.feature import StringIndexer
indexer=StringIndexer(inputCol="Cruise_line",outputCol="index").fit(df)
da=indexer.transform(df)
da.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|index|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7| 1.0|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1| 1.0|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0| 1.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2| 1.0|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5| 1.0|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6| 1.0|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3| 1.0|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3| 1.0|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3| 1.0|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----+
only showing top 20 rows

In [7]:
assembler=VectorAssembler(inputCols=['Age','Tonnage','passengers','length','cabins','passenger_density','index'],outputCol='features')
output=assembler.transform(da)

In [8]:
output.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)
-- index: double (nullable = false)
-- features: vector (nullable = true)

In [9]:
output.select('features').show()

+--------------------+
 features|
+--------------------+
[6.0,30.276999999...|
[6.0,30.276999999...|
[26.0,47.262,14.8...|
[11.0,110.0,29.74...|
[17.0,101.353,26....|
[22.0,70.367,20.5...|
[15.0,70.367,20.5...|
[23.0,70.367,20.5...|
[19.0,70.367,20.5...|
[6.0,110.23899999...|
[10.0,110.0,29.74...|
[28.0,46.052,14.5...|
[18.0,70.367,20.5...|
[17.0,70.367,20.5...|
[11.0,86.0,21.24,...|
[8.0,110.0,29.74,...|
[9.0,88.5,21.24,9...|
[15.0,70.367,20.5...|
[12.0,88.5,21.24,...|
[20.0,70.367,20.5...|
+--------------------+
only showing top 20 rows

In [10]:
final=output.select('crew','features')

In [11]:
train,test=final.randomSplit([0.68,0.32])

In [12]:
lr=LinearRegression(labelCol='crew')

In [13]:
model=lr.fit(train)
res=model.evaluate(test)

In [14]:
res.residuals.show()

+--------------------+
 residuals|
+--------------------+
 -0.7209653555382894|
 0.3133103806901141|
-0.05344435583345453|
0.002473513630618651|
 -0.9933931299632086|
 -0.8949355245756356|
 0.21035915963979068|
-0.18607363658181963|
 0.0734053481985919|
 1.7810172353582643|
 0.8296613410572857|
 -1.1260743248098164|
 -0.5571823648672378|
-0.23467626289672872|
 -1.351377913151972|
 -1.3327386233306138|
 0.12393654125422682|
 0.8595562741359535|
-0.03609219177228784|
 -1.1243211206656927|
+--------------------+
only showing top 20 rows

In [15]:
res.r2

Out[ 35 ]: 0.9481274704113987

In [16]:
res.rootMeanSquaredError

Out[ 36 ]: 0.7396336363325648

In [17]:
res.meanAbsoluteError

Out[ 37 ]: 0.6162469399301522

In [18]:
res.meanSquaredError

Out[ 38 ]: 0.5470579159945328

In [19]:
from pyspark.sql.functions import corr

In [20]:
df.select(corr('crew','passengers')).show()

+----------------------+
corr(crew, passengers)|
+----------------------+
 0.9152341306065384|
+----------------------+